In [1]:
import os
os.chdir('..')

from seg_game_typehinted import SevenSegButtonGame, MiniGame
from rpi_tm1638_animations import TM1638Animated as Tm

In [2]:
from random import randint, shuffle
from typing import Any, Dict, List

In [3]:
space_win_length = 4
fragment_nums = []


def space_setup(tm1638: Tm) -> Dict[str, Any]:
    """
    Setup answers and starting display for space game
    """
    # set the win length to be half of the number of segments
    space_win_length = int(tm1638.num_segments / 2)
    
    # generate the fragments
    fragment_nums = [randint(1, 127) for i in range(space_win_length)]
    counterpart_fragments = [127 - frag for frag in fragment_nums]
    
    # randomise the fragment order
    counterpart_shuffle = list(range(4))
    shuffle(counterpart_shuffle)

    for i in range(len(counterpart_shuffle)):
        place = counterpart_shuffle.index(i)
        fragment_nums.append(counterpart_fragments[place])
    
    globals()['fragment_nums'] = fragment_nums

    
    correct_answer_sequence = [i+4 for i in counterpart_shuffle]
    
    
    return_dict = {
        'correct_answer_conditions': correct_answer_sequence,
        'game_seg_display': fragment_nums,
        'game_LED_display': 1 << 8
    }
        
    return return_dict


def space_correct_answer_action(tm1638: Tm,
                                progress: int) -> List[int]:
    """
    Update display after correct answer given
    """
    fragment_nums[progress - 1] = 127
    globals()['fragment_nums'] = fragment_nums
    
    tm1638.LEDs(1 << 8 - progress)
    
    
    return fragment_nums


space_game = MiniGame(win_length=space_win_length,
                      setup_routine=space_setup,
                      correct_answer_action=space_correct_answer_action,
                      input_as_linear_int=True)

seg_game = SevenSegButtonGame(stb=2,
                              clk=3,
                              dio=4,
                              test_mode=True)
seg_game.register_game('space', space_game)
seg_game.setup()

In [4]:
# Check math game answers (all power of 2)
space_game.correct_answer_conditions

[7, 5, 4, 6]

In [5]:
# Check the start display
space_game.game_seg_display

[81, 80, 115, 18, 12, 47, 109, 46]

In [6]:
def test_play(input):
    print(f'playing: {input}')
    space_game.play(1 << input)

In [7]:
test_play(0)

playing: 0
1 converted to: 0
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.
 *   •   •   •   •   •   •   •   •  
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.


In [8]:
# Check all answers correct wins game
for answer in space_game.correct_answer_conditions:
    test_play(answer)

playing: 7
128 converted to: 7
 *   •   •   •   •   •   •   •  
 *   •   •   •   •   •   •   •   •  
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.
playing: 5
32 converted to: 5
 •   *   •   •   •   •   •   •  
 *   •   •   •   •   •   •   •   •  
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.
playing: 4
16 converted to: 4
 •   •   *   •   •   •   •   •  
 *   •   •   •   •   •   •   •   •  
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.
playing: 6
64 converted to: 6
 •   •   •   *   •   •   •   •  
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.


In [9]:
# Check the game stays finished
test_play(0)

playing: 0
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.
